In [1]:
%cd ../..
%load_ext autoreload
%autoreload 2

/root/capsule/code


In [2]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
from pipeline import lab, experiment, ephys, psth_foraging, foraging_analysis

foraging_sessions = (foraging_analysis.SessionTaskProtocol & 'session_task_protocol=100').proj()
import datajoint as dj; dj.conn().connect()

[2023-03-30 02:32:59,110][INFO]: Connecting map-superuser@datajoint.mesoscale-activity-map.org:3306
[2023-03-30 02:32:59,680][INFO]: Connected map-superuser@datajoint.mesoscale-activity-map.org:3306


# Add choices and autoregression into linear regression

In [6]:
psth_foraging.UnitPeriodLinearFit.progress()

UnitPeriodLinearFit  Completed 714777 of 714777 (100.0%)   2023-03-29 19:08:31


(0, 714777)

In [15]:
psth_foraging.IndependentVariable()

var_name,desc
choice_ic,ipsi (0) or contra (1)
choice_ic_next,"ipsi (0) or contra (1), next choice"
choice_lr,left (0) or right (1)
contra_action_value,contra action_value
contra_choice_kernel,contra choice_kernel
contra_choice_prob,contra choice_prob
firing_1_back,firing rate of 1 trials back
firing_2_back,firing rate of 2 trials back
firing_3_back,firing rate of 3 trials back
firing_4_back,firing rate of 4 trials back


In [26]:
psth_foraging.LinearModel.load()
psth_foraging.LinearModel()

multi_linear_model,if_intercept Whether intercept is included
"dQ, sumQ, rpe, C*2",1
"dQ, sumQ, rpe, C*2, R*1",1
"dQ, sumQ, rpe, C*2, R*1, t",1
"dQ, sumQ, rpe, C*2, R*5, t",1
"dQ, sumQ, rpe, C*2, t",1
Q_c + Q_i + rpe,1
Q_l + Q_r + rpe,1
Q_rel + Q_tot + rpe,1


## Debug fitting

In [27]:
date, imec, unit = '2021-04-18', 0, 541
unit_key = (ephys.Unit() & (experiment.Session & 'session_date = "2021-04-18"' & 'subject_id = 473361') & {'insertion_number': imec + 1, 'unit_uid': unit}).fetch1("KEY")
key = {**unit_key, 'period': "iti_all", 'multi_linear_model': "dQ, sumQ, rpe, C*2"}    

In [39]:
key = {'subject_id': 473360, 'session': 49, 'insertion_number': 1, 'clustering_method': 'kilosort2', 'unit': 187, 'period': 'delay', 'behavior_model': 'best_aic', 'multi_linear_model': 'dQ, sumQ, rpe, C*2'}

In [3]:
psth_foraging.UnitPeriodLinearFit.delete()

[2023-03-30 01:26:39,065][INFO]: Deleting 93632 rows from `map_v2_psth_foraging`.`__unit_period_linear_fit__param`
[2023-03-30 01:26:39,806][INFO]: Deleting 17024 rows from `map_v2_psth_foraging`.`__unit_period_linear_fit`


Deletes committed.


17024

In [28]:
len(psth_foraging.UnitPeriodLinearFit())

17304

In [31]:
psth_foraging.UnitPeriodLinearFit & 'period = "iti_all"'

subject_id institution 6 digit animal ID,session session number,insertion_number,clustering_method,unit,period,"behavior_model If 'best_aic' etc, the best behavioral model for each session; otherwise --> Model.model_id",multi_linear_model,actual_behavior_model,model_r2 r square,model_r2_adj r square adj.,model_p,model_bic,model_aic
473361,47,1,kilosort2,541,iti_all,best_aic,"dQ, sumQ, rpe, C*2",16,0.579215,0.574134,0.0,2892.07,2867.83
473361,47,1,kilosort2,541,iti_all,best_aic,"dQ, sumQ, rpe, C*2, R*1",16,0.63542,0.630111,0.0,2831.58,2803.32
473361,47,1,kilosort2,541,iti_all,best_aic,"dQ, sumQ, rpe, C*2, R*1, t",16,0.66198,0.656223,0.0,2805.93,2773.63
473361,47,1,kilosort2,541,iti_all,best_aic,"dQ, sumQ, rpe, C*2, R*5, t",16,0.662665,0.653457,0.0,2804.22,2755.88
473361,47,1,kilosort2,541,iti_all,best_aic,"dQ, sumQ, rpe, C*2, t",16,0.634914,0.62961,0.0,2838.48,2810.19
473361,47,1,kilosort2,541,iti_all,best_aic,Q_c + Q_i + rpe,16,0.395207,0.390856,2.8026e-45,3038.52,3022.35
473361,47,1,kilosort2,541,iti_all,best_aic,Q_l + Q_r + rpe,16,0.395207,0.390856,2.8026e-45,3038.52,3022.35
473361,47,1,kilosort2,541,iti_all,best_aic,Q_rel + Q_tot + rpe,16,0.395207,0.390856,2.8026e-45,3038.52,3022.35


In [75]:
psth_foraging.schema.jobs.delete()

In [78]:
dj.U('host').aggr(psth_foraging.schema.jobs, n='COUNT(*)')

host system hostname,n calculated attribute


In [27]:
psth_foraging.schema.jobs

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__unit_period_linear_fit,00dd0a37a177fa949168dc63da2b8109,reserved,=BLOB=,,=BLOB=,map-superuser@ec2-44-231-51-170.us-west-2.compute.amazonaws.com,da690d313936,30151,22189520,2023-03-30 02:46:13
__unit_period_linear_fit,0183fc9c04a6c82f4e6b1c5a75bb3ac8,reserved,=BLOB=,,=BLOB=,map-superuser@ec2-44-231-51-170.us-west-2.compute.amazonaws.com,da690d313936,30112,22189520,2023-03-30 02:46:13
__unit_period_linear_fit,01f7b0dbfad97d1e64e8fa20881bbe0a,reserved,=BLOB=,,=BLOB=,map-superuser@ec2-44-231-51-170.us-west-2.compute.amazonaws.com,da690d313936,30068,22189520,2023-03-30 02:46:13
__unit_period_linear_fit,059b54f0dc5a0ef6a396270696703aef,reserved,=BLOB=,,=BLOB=,map-superuser@ec2-44-231-51-170.us-west-2.compute.amazonaws.com,da690d313936,30141,22189520,2023-03-30 02:46:13
__unit_period_linear_fit,05bf81a873d00fc31913496ff4f532cd,reserved,=BLOB=,,=BLOB=,map-superuser@ec2-44-231-51-170.us-west-2.compute.amazonaws.com,da690d313936,30124,22189520,2023-03-30 02:46:13
__unit_period_linear_fit,0b03fe88ff355db636ef222d1a9d9d97,reserved,=BLOB=,,=BLOB=,map-superuser@ec2-44-231-51-170.us-west-2.compute.amazonaws.com,da690d313936,30144,22189520,2023-03-30 02:46:13
__unit_period_linear_fit,0c5d40ee67b7ffa8d613165008638cde,reserved,=BLOB=,,=BLOB=,map-superuser@ec2-44-231-51-170.us-west-2.compute.amazonaws.com,da690d313936,29716,22189192,2023-03-30 02:46:13
__unit_period_linear_fit,0cd6015d3540b4d97dabe9533bd91aff,reserved,=BLOB=,,=BLOB=,map-superuser@ec2-44-231-51-170.us-west-2.compute.amazonaws.com,da690d313936,29714,22189192,2023-03-30 02:46:13
__unit_period_linear_fit,138eec6a6531b685a2093a90f7912e98,reserved,=BLOB=,,=BLOB=,map-superuser@ec2-44-231-51-170.us-west-2.compute.amazonaws.com,da690d313936,30076,22189520,2023-03-30 02:46:13
__unit_period_linear_fit,17f7c1a1fb00e79f41e1ff2d66c33e64,reserved,=BLOB=,,=BLOB=,map-superuser@ec2-44-231-51-170.us-west-2.compute.amazonaws.com,da690d313936,30074,22189520,2023-03-30 02:46:13
